In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
basedir="/Users/tgadf/Downloads/osm"

Notebook Last Run Initiated: 2018-09-19 16:39:45.066976


In [6]:
from modelio import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

fastfood=["fastfood", "foodcourt","convenience", "cafe"]
restaurant=["restaurant"]
homefood=["greengrocer", "pharmacy", "supermarket", "chemist"]
lodging=["hotel", "motel", "hostel","chalet","bedandbreakfast","guesthouse", "caravansite"]
commercial=["bookshop","butcher","hairdresser", "furnitureshop", "doityourself", "gardencentre", "newsagent", "toyshop","mall","laundry","mobilephoneshop","bank","atm"]
commercial+=["bicyclerental","bicycleshop","bakery","beautyshop","videoshop","outdoorshop", 'jeweller','kiosk', 'shoeshop','sportsshop', "travelagent"]
commercial+=['florist','giftshop', "departmentstore", "computershop"]
vending=['vendingany','vendingcigarette','vendingmachine','vendingparking']
commercial+=vending
auto=['cardealership','carrental','carrepair','carsharing','carwash']
municipal=["firestation", "prison", "library", "postoffice", "postbox", "police", "townhall","communitycentre","artscentre", "publicbuilding", "courthouse"]
industrial=["wastewaterplant", "windmill","watermill","waterworks", "recycling", "recyclingclothes", "recyclingglass", "recyclingmetal", "recyclingpaper", "commstower", 'watertower','waterwell', "tower"]
building=["embassy"]
medical=["hospital", "dentist", "doctors", "optician", "veterinary","nursinghome"]
attraction=["attraction","fountain", "museum", "touristinfo", "viewpoint", "zoo","themepark","monument","picnicsite","fort","castle","battlefield","archaeological","memorial", "lighthouse", "observationtower", "artwork"]
religious=["graveyard","waysidecross","waysideshrine","ruins"]
sport=["golfcourse", "huntingstand", "icerink", "sportscentre", "track", "swimmingpool"]
entertainment=["bar", "pub", "nightclub", "theatre", "cinema"]
recreation=["stadium", "campsite","biergarten","dogpark","park", "pitch", "playground"]
school=["kindergarten","school"]
college=["college","university"]
general=['alpinehut','bench','beverages','camerasurveillance','clothes','drinkingwater','shelter','stationery','telephone','toilet','wastebasket']                

increase={}

def parseOSM(dirval, prec, key=""):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*{0}*.shp".format(key)))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    
    
    shapeData   = {}
    geoShapeMap = {}
    ngeos     = 0
    totalgeos = 0
    show      = False
    cntr      = Counter()
        
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, prec)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        Nshapes   = len(sf.shapes())

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 2500 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                if fclass.find("christian_") != -1:
                    fclass = "christian"
                elif fclass.find("muslim_") != -1:
                    fclass = "muslim"
                else:
                    fclass = fclass
                geoid = fclass
            elif osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("bicycle") != -1:
                    fclass = "bicycle"
                elif fclass.find("parking_") != -1:
                    fclass = "parking"
                elif fclass.find("mini_roundabout") != -1 or fclass.find("turning_circle") != -1:
                    fclass = "roundabout"
                elif fclass.find("speed_camera") != -1:
                    fclass = "speedcamera"
                elif fclass.find("motorway_junction") != -1:
                    fclass = "ramp"
                elif fclass.find("lock_gate") != -1:
                    fclass = "gate"
                elif fclass.find("traffic_signals") != -1:
                    fclass = "signal"
                elif fclass.find("street_lamp") != -1:
                    fclass = "lamp"
                else:
                    fclass = fclass
                    
                if fclass not in ["parking","fuel"]:
                    continue
                geoid = fclass
            elif osm in ["gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("railway_") != -1:
                    fclass = fclass.replace("railway", "rail")
                fclass = fclass.replace("_", "")
                
                if fclass.startswith("bus"):
                    fclass = "bus"
                elif fclass.startswith("rail"):
                    fclass = "rail"
                elif fclass.startswith("ferry"):
                    fclass = "ferry"
                elif fclass.startswith("tram"):
                    fclass = "tram"
                elif fclass.startswith("taxi"):
                    fclass = "taxi"
                else:
                    raise ValueError("fClass {0} was not recognized!".format(fclass))
                
                geoid = fclass
            elif osm in ["gis_osm_pois_a_free_1", "gis_osm_pois_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                fclass = fclass.replace("_", "")
                
                if fclass in fastfood:
                    fclass = "fastfood"
                elif fclass in restaurant:
                    fclass = "restaurant"
                elif fclass in homefood:
                    fclass = "grocery"
                elif fclass in lodging:
                    fclass = "lodging"
                elif fclass in commercial:
                    fclass = "commercial"
                elif fclass in auto:
                    fclass = "auto"
                elif fclass in municipal:
                    fclass = "municipal"
                elif fclass in industrial:
                    fclass = "industrial"
                elif fclass in building:
                    fclass = "building"
                elif fclass in medical:
                    fclass = "medical"
                elif fclass in attraction:
                    fclass = "attraction"
                elif fclass in religious:
                    fclass = "religious"
                elif fclass in sport:
                    fclass = "sport"
                elif fclass in entertainment:
                    fclass = "entertainment"
                elif fclass in recreation:
                    fclass = "recreation"
                elif fclass in school:
                    fclass = "school"
                elif fclass in college:
                    fclass = "college"
                elif fclass in general:
                    fclass = "public"
                else:
                    raise ValueError("fClass {0} was not found!".format(fclass))
                geoid = fclass
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #if irec > 1000: 1/0

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            useLinear = False
            if len(points) == 1 or useLinear is True:
                useLinear = True
                geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
            else:
                geos = getInitGeo(shape)
                geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                if geos is None:
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = 1
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        print("Finished: {0}".format(cntr.most_common(100)))
        print("\n")
            
    found = False
    for k,v in cntr.most_common():
        if k.find(" ") != -1 or k.find(";") != -1 or k[0].isupper():
            print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
            found = True
    print("\n")
    for k,v in nameCntr.most_common(200):
        print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
    if len(shapeData) > 0:
        for geoid in geoShapeMap.keys():
            geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
        print("Found {0} geos from {1}".format(ngeos, shapeval))
        saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, key, prec)))

In [14]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    if exists(dirval):
        print("Directory {0} already exists".format(dirval))
        rmZipDir(dirval)
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    print("Extract {0}".format(zipname))
    zip_ref.extractall(dirval)
    zip_ref.close()
    
    parseOSM(dirval, 7, 'pois')
    
    rmZipDir(dirval)

Extract /Users/tgadf/Downloads/osm/alabama-latest-free.shp.zip
gis_osm_pois_a_free_1
Current Time is Wed Sep 19, 2018 18:36:23 for 

Analyzing gis_osm_pois_a_free_1	11525
Processed 2500/11525 records. Found 10375 geos so far...
Processed 5000/11525 records. Found 18926 geos so far...
Processed 7500/11525 records. Found 25959 geos so far...
Processed 10000/11525 records. Found 34901 geos so far...
Finished: [('recreation', 5356), ('sport', 1265), ('religious', 590), ('commercial', 550), ('school', 547), ('public', 482), ('fastfood', 473), ('restaurant', 455), ('municipal', 432), ('attraction', 256), ('lodging', 240), ('grocery', 225), ('medical', 216), ('industrial', 155), ('auto', 134), ('entertainment', 89), ('college', 60)]


gis_osm_pois_free_1
Current Time is Wed Sep 19, 2018 18:36:45 for 

Analyzing gis_osm_pois_free_1	11548
Processed 2500/11548 records. Found 41020 geos so far...
Processed 5000/11548 records. Found 43520 geos so far...
Processed 7500/11548 records. Found 46020 ge

Processed 37500/125683 records. Found 145843 geos so far...
Processed 40000/125683 records. Found 153028 geos so far...
Processed 42500/125683 records. Found 158974 geos so far...
Processed 45000/125683 records. Found 165105 geos so far...
Processed 47500/125683 records. Found 171667 geos so far...
Processed 50000/125683 records. Found 178368 geos so far...
Processed 52500/125683 records. Found 184353 geos so far...
Processed 55000/125683 records. Found 189587 geos so far...
Processed 57500/125683 records. Found 198402 geos so far...
Processed 60000/125683 records. Found 203321 geos so far...
Processed 62500/125683 records. Found 208822 geos so far...
Processed 65000/125683 records. Found 214599 geos so far...
Processed 67500/125683 records. Found 219431 geos so far...
Processed 70000/125683 records. Found 224494 geos so far...
Processed 72500/125683 records. Found 229182 geos so far...
Processed 75000/125683 records. Found 234326 geos so far...
Processed 77500/125683 records. Found 23

Extract /Users/tgadf/Downloads/osm/connecticut-latest-free.shp.zip
gis_osm_pois_a_free_1
Current Time is Wed Sep 19, 2018 18:43:18 for 

Analyzing gis_osm_pois_a_free_1	10260
Processed 2500/10260 records. Found 16960 geos so far...
Processed 5000/10260 records. Found 26633 geos so far...
Processed 7500/10260 records. Found 34377 geos so far...
Processed 10000/10260 records. Found 41179 geos so far...
Finished: [('recreation', 4586), ('religious', 1417), ('school', 1018), ('sport', 848), ('commercial', 344), ('municipal', 342), ('fastfood', 331), ('restaurant', 310), ('public', 188), ('grocery', 153), ('attraction', 150), ('lodging', 131), ('auto', 127), ('medical', 118), ('entertainment', 78), ('industrial', 62), ('college', 57)]


gis_osm_pois_free_1
Current Time is Wed Sep 19, 2018 18:43:41 for 

Analyzing gis_osm_pois_free_1	12300
Processed 2500/12300 records. Found 44644 geos so far...
Processed 5000/12300 records. Found 47144 geos so far...
Processed 7500/12300 records. Found 4964

gis_osm_pois_a_free_1
Current Time is Wed Sep 19, 2018 18:46:21 for 

Analyzing gis_osm_pois_a_free_1	22015
Processed 2500/22015 records. Found 20915 geos so far...
Processed 5000/22015 records. Found 27286 geos so far...
Processed 7500/22015 records. Found 34747 geos so far...
Processed 10000/22015 records. Found 43437 geos so far...
Processed 12500/22015 records. Found 49276 geos so far...
Processed 15000/22015 records. Found 57143 geos so far...
Processed 17500/22015 records. Found 65370 geos so far...
Processed 20000/22015 records. Found 72883 geos so far...
Finished: [('recreation', 9947), ('sport', 2455), ('religious', 1484), ('fastfood', 1419), ('restaurant', 1055), ('commercial', 924), ('municipal', 825), ('school', 763), ('public', 586), ('lodging', 506), ('college', 470), ('attraction', 401), ('grocery', 388), ('medical', 249), ('auto', 221), ('industrial', 194), ('entertainment', 128)]


gis_osm_pois_free_1
Current Time is Wed Sep 19, 2018 18:47:07 for 

Analyzing gis_osm_po

Processed 7500/18827 records. Found 26213 geos so far...
Processed 10000/18827 records. Found 35453 geos so far...
Processed 12500/18827 records. Found 43789 geos so far...
Processed 15000/18827 records. Found 50107 geos so far...
Processed 17500/18827 records. Found 55715 geos so far...
Finished: [('recreation', 10086), ('sport', 2224), ('public', 803), ('religious', 772), ('fastfood', 721), ('commercial', 707), ('school', 688), ('restaurant', 579), ('municipal', 575), ('lodging', 321), ('attraction', 285), ('grocery', 273), ('medical', 224), ('auto', 198), ('industrial', 164), ('entertainment', 130), ('college', 77)]


gis_osm_pois_free_1
Current Time is Wed Sep 19, 2018 18:49:20 for 

Analyzing gis_osm_pois_free_1	12455
Processed 2500/12455 records. Found 61426 geos so far...
Processed 5000/12455 records. Found 63926 geos so far...
Processed 7500/12455 records. Found 66426 geos so far...
Processed 10000/12455 records. Found 68926 geos so far...
Finished: [('recreation', 11084), ('sc

...

Processed 40000/49592 records. Found 125535 geos so far...
Processed 42500/49592 records. Found 130784 geos so far...
Processed 45000/49592 records. Found 136799 geos so far...
Processed 47500/49592 records. Found 143811 geos so far...
Finished: [('recreation', 24173), ('sport', 6691), ('fastfood', 2552), ('school', 2469), ('commercial', 2135), ('public', 1808), ('religious', 1731), ('restaurant', 1717), ('municipal', 1523), ('grocery', 823), ('auto', 762), ('attraction', 752), ('lodging', 749), ('industrial', 563), ('medical', 498), ('entertainment', 469), ('college', 177)]


gis_osm_pois_free_1
Current Time is Wed Sep 19, 2018 19:06:48 for 

Analyzing gis_osm_pois_free_1	28566
Processed 2500/28566 records. Found 151215 geos so far...
Processed 5000/28566 records. Found 153715 geos so far...
Processed 7500/28566 records. Found 156215 geos so far...
Processed 10000/28566 records. Found 158715 geos so far...
Processed 12500/28566 records. Found 161215 geos so far...
Processed 15000/2856

Finished: [('recreation', 19425), ('school', 7057), ('commercial', 6940), ('restaurant', 5558), ('public', 5499), ('sport', 5308), ('municipal', 4786), ('fastfood', 4586), ('attraction', 2980), ('religious', 2278), ('industrial', 2083), ('grocery', 1929), ('entertainment', 1552), ('auto', 1392), ('lodging', 1223), ('medical', 1179), ('college', 356), ('building', 2)]




Found 159752 geos from gis_osm_pois_free_1


There are 18 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/pennsylvania-pois-7-data.p


There are 159752 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/pennsylvania-pois-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/pennsylvania-latest-free.shp
Extract /Users/tgadf/Downloads/osm/puerto-rico-latest-free.shp.zip
gis_osm_pois_a_free_1
Current Time is Wed Sep 19, 2018 19:09:52 for 

Analyzing gis_osm_pois_a_free_1	8459
Processed 2500/8459 records. Found 6239 geos so far...
Processed 5000/8459 records. Found 11511 g

Processed 5000/14005 records. Found 55054 geos so far...
Processed 7500/14005 records. Found 57554 geos so far...
Processed 10000/14005 records. Found 60054 geos so far...
Processed 12500/14005 records. Found 62554 geos so far...
Finished: [('school', 6386), ('recreation', 6189), ('restaurant', 1708), ('sport', 1660), ('commercial', 1479), ('municipal', 1439), ('fastfood', 1414), ('attraction', 1008), ('public', 1000), ('industrial', 957), ('religious', 800), ('lodging', 754), ('grocery', 660), ('medical', 435), ('entertainment', 326), ('auto', 304), ('college', 114)]




Found 64059 geos from gis_osm_pois_free_1


There are 17 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/tennessee-pois-7-data.p


There are 64059 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/tennessee-pois-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/tennessee-latest-free.shp
Extract /Users/tgadf/Downloads/osm/texas-latest-free.shp.zip
gis_osm_pois_a_f

Processed 2500/30354 records. Found 11715 geos so far...
Processed 5000/30354 records. Found 25452 geos so far...
Processed 7500/30354 records. Found 31523 geos so far...
Processed 10000/30354 records. Found 37645 geos so far...
Processed 12500/30354 records. Found 45090 geos so far...
Processed 15000/30354 records. Found 52599 geos so far...
Processed 17500/30354 records. Found 61291 geos so far...
Processed 20000/30354 records. Found 68081 geos so far...
Processed 22500/30354 records. Found 73956 geos so far...
Processed 25000/30354 records. Found 84045 geos so far...
Processed 27500/30354 records. Found 102473 geos so far...
Processed 30000/30354 records. Found 109028 geos so far...
Finished: [('recreation', 14887), ('sport', 3048), ('fastfood', 1646), ('school', 1600), ('commercial', 1588), ('restaurant', 1413), ('municipal', 965), ('lodging', 895), ('public', 854), ('religious', 745), ('attraction', 707), ('grocery', 631), ('auto', 423), ('medical', 358), ('industrial', 271), ('en

Current Time is Wed Sep 19, 2018 19:20:29 for 

Analyzing gis_osm_pois_free_1	22484
Processed 2500/22484 records. Found 117188 geos so far...
Processed 5000/22484 records. Found 119688 geos so far...
Processed 7500/22484 records. Found 122188 geos so far...
Processed 10000/22484 records. Found 124688 geos so far...
Processed 12500/22484 records. Found 127188 geos so far...
Processed 15000/22484 records. Found 129688 geos so far...
Processed 17500/22484 records. Found 132188 geos so far...
Processed 20000/22484 records. Found 134688 geos so far...
Finished: [('recreation', 19159), ('public', 4836), ('school', 4217), ('commercial', 3236), ('fastfood', 2884), ('attraction', 2574), ('restaurant', 2554), ('municipal', 2230), ('sport', 2204), ('religious', 2130), ('industrial', 1909), ('entertainment', 1332), ('grocery', 841), ('lodging', 755), ('auto', 610), ('medical', 526), ('college', 164), ('building', 1)]




Found 137172 geos from gis_osm_pois_free_1


There are 18 entries in the save